<a href="https://colab.research.google.com/github/star14336/Clinical_LLM/blob/main/KAIA_LLM_FT_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KAIA 2024 Spring
## 나만의 의료 인공지능 만들기

- 연자: 김준우(kjune0322@kaist.ac.kr), 권순준(sean0042@kaist.ac.kr)
- 발표자료: https://github.com/starmpcc/KAIA-LLM-FT-2024




<a target="_blank" href="https://colab.research.google.com/github/starmpcc/KAIA-LLM-FT-2024/blob/main/KAIA_LLM_FT.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
# First, install required packages
!pip install -q accelerate==0.25.0 peft==0.6.2 bitsandbytes==0.41.1 transformers==4.36.2 trl==0.7.4 einops gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 11.9 MB

In [2]:
# Import Libraries
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
import gradio as gr

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [3]:
# To save time, first download model and data

# Define Quantization Config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
)

# Load Model and Dataset
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-2",
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map="auto",
    revision="refs/pr/23"
)

tokenizer = AutoTokenizer.from_pretrained('microsoft/phi-2')
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_sight = "right"

def prompt_shorter_than(samples):
    concatenated = [" ".join([i, j, k]) for i, j, k in zip(samples['note'], samples['question'], samples['answer'])]
    return [len(i)<=320 for i in tokenizer(concatenated)['input_ids']]

dataset = load_dataset("starmpcc/Asclepius-Synthetic-Clinical-Notes")
dataset = dataset.filter(lambda x: [len(i)<1500 for i in x['note']], batched=True)
dataset = dataset.filter(prompt_shorter_than, batched=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

modeling_phi.py:   0%|          | 0.00/33.7k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.3k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Generating train split:   0%|          | 0/158114 [00:00<?, ? examples/s]

Filter:   0%|          | 0/158114 [00:00<?, ? examples/s]

Filter:   0%|          | 0/36654 [00:00<?, ? examples/s]

In [4]:
# Let's pre-process dataset

# First of all, we have to check how the dataset is composed
print(dataset['train'])
dataset['train'][0]

Dataset({
    features: ['patient_id', 'note', 'question', 'answer', 'task'],
    num_rows: 3686
})


{'patient_id': 110,
 'note': "Hospital Course:\n\nThe patient, who was involved in a murder case, was admitted to our forensic facility for toxicology testing and a complete neuropsychiatric evaluation. The case is complicated due to the defendant's claimed genetic predisposition to anti-social behavior and his regular consumption of alcohol and drugs since the beginning of adolescence.\n\nToxicology testing revealed detectable levels of benzoylecgonine in urine and pubic hair, while blood and saliva samples showed no significant levels of drugs or alcohol. A full clinical and neuropsychological examination was performed, which identified a personality disorder not otherwise specified. MRI imaging showed a decrease in cortical thickness with enlarged lateral ventricles, significant volumetric asymmetry of the amygdalae, and a decreased volume of the right orbito-frontal cortex in comparison with the left one. PET-CT testing did not indicate any alteration of brain perfusion or metaboli

In [20]:
# We make this dataset to phi-2 compatible
# Phi-2 instruction-answer format: "Instruct: <prompt>\nOutput:"

# Make your own prompt!
prompt_template="""Instruct:
You are a intelligent doctor K. I give you the clinical note as {{note}}
{note}
You should answer to {{question}} based on the clinical note. I expect you can give me accurate answer concisely.
{question}
You should maintain the phi-2 format
Give me Output on last line.

Output: {answer}
"""

# Should get Dict[List] as input, return list of prompts
def format_dataset(samples):
    outputs = []
    for _, note, question, answer, _ in zip(*samples.values()):
        out = prompt_template.format(note=note, question=question, answer=answer)
        outputs.append(out)
    return outputs

sample_input = format_dataset({k: [v] for k, v in dataset['train'][0].items()})[0]
print(sample_input)
print("*"*20)

# Sanity Check
prompt_len = len(tokenizer.encode(prompt_template))
if prompt_len > 180:
    raise ValueError(f"Your prompt is too long! Please reduce the length from {prompt_len} to 180 tokens")
print(f"Prompt Length: {prompt_len} tokens")

Instruct:
You are a intelligent doctor K. I give you the clinical note as {note}
Hospital Course:

The patient, who was involved in a murder case, was admitted to our forensic facility for toxicology testing and a complete neuropsychiatric evaluation. The case is complicated due to the defendant's claimed genetic predisposition to anti-social behavior and his regular consumption of alcohol and drugs since the beginning of adolescence.

Toxicology testing revealed detectable levels of benzoylecgonine in urine and pubic hair, while blood and saliva samples showed no significant levels of drugs or alcohol. A full clinical and neuropsychological examination was performed, which identified a personality disorder not otherwise specified. MRI imaging showed a decrease in cortical thickness with enlarged lateral ventricles, significant volumetric asymmetry of the amygdalae, and a decreased volume of the right orbito-frontal cortex in comparison with the left one. PET-CT testing did not indicat

In [21]:
sample_idx = 0
sample_input = format_dataset({k: [v] for k, v in dataset['train'][sample_idx].items()})[0].split('Output: ')[0]
input_ids = tokenizer.encode(sample_input, return_tensors='pt').to('cuda')
with torch.no_grad():
  output = model.generate(input_ids=input_ids,
                            max_length=512,
                            use_cache=True,
                            temperature=0.,
                            eos_token_id=tokenizer.eos_token_id
  )
print(tokenizer.decode(output.to('cpu')[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Instruct:
You are a intelligent doctor K. I give you the clinical note as {note}
Hospital Course:

The patient, who was involved in a murder case, was admitted to our forensic facility for toxicology testing and a complete neuropsychiatric evaluation. The case is complicated due to the defendant's claimed genetic predisposition to anti-social behavior and his regular consumption of alcohol and drugs since the beginning of adolescence.

Toxicology testing revealed detectable levels of benzoylecgonine in urine and pubic hair, while blood and saliva samples showed no significant levels of drugs or alcohol. A full clinical and neuropsychological examination was performed, which identified a personality disorder not otherwise specified. MRI imaging showed a decrease in cortical thickness with enlarged lateral ventricles, significant volumetric asymmetry of the amygdalae, and a decreased volume of the right orbito-frontal cortex in comparison with the left one. PET-CT testing did not indicat

In [22]:
# Then, let's define dataset.
response_template = "Output:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

train_dataset = dataset['train']
sampled_train_dataset = train_dataset.select(range(2000))

In [23]:
# SFTTrainer Do everything else for you!

lora_config=LoraConfig(
    r=4,
    task_type="CAUSAL_LM",
    target_modules= ["Wqkv", "fc1", "fc2" ]
)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    fp16=True,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    learning_rate=1e-4,
    optim="paged_adamw_32bit",
    save_strategy="no",
    warmup_ratio=0.03,
    logging_steps=5,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
    gradient_checkpointing=True
)

trainer = SFTTrainer(
    model,
    training_args,
    train_dataset=sampled_train_dataset,
    formatting_func=format_dataset,
    data_collator=collator,
    peft_config=lora_config,
    max_seq_length=512,
    tokenizer=tokenizer,
)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
# Run Training
trainer.train()

You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


Step,Training Loss
5,0.454100
10,0.470500
15,0.476200
20,0.452000
25,0.403300
30,0.398400
35,0.443000
40,0.537000
45,0.497100
50,0.451500


In [19]:
# Wrap-up Training
model = trainer.model
model.eval()

note_samples = train_dataset.select(range(len(train_dataset)-10, len(train_dataset)))['note']

def inference(note, question, model):
    prompt = prompt_template.format(note=note, question=question, answer="")
    tokens = tokenizer.encode(prompt, return_tensors="pt").to('cuda')
    outs = model.generate(input_ids=tokens,
                          max_length=512,
                          use_cache=True,
                          temperature=0.,
                          eos_token_id=tokenizer.eos_token_id
                          )
    output_text = tokenizer.decode(outs.to('cpu')[0], skip_special_tokens=True)
    return output_text[len(prompt):]


def compare_models(note, question):
    with torch.no_grad():
        asc_answer = inference(note, question, trainer.model)
        with model.disable_adapter():
            phi_answer = inference(note, question, trainer.model)
    return asc_answer, phi_answer

demo = gr.Interface(fn=compare_models, inputs=[gr.Dropdown(note_samples), "text"], outputs=[gr.Textbox(label="Asclepius"), gr.Textbox(label="Phi-2")])
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://847c093a5e313eff52.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
